## BERT 嵌入生成

安装Transformers库，这里使用的版本是3.5.1

In [ ]:
! pip install Transformers==3.5.1

### BERT的顶层编码器（编码器12）获得嵌入

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

下载预训练的BERT模型，使用的是不区分大小写的模型

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')  # 下载并加载预训练模型
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 下载并加载用于预训练模型的词元分析器

#### 输入预处理

In [ ]:
sentence = 'I love Paris'
# 对句子进行分词
tokens = tokenizer.tokenize(sentence)
print(tokens)
# 添加[CLS]和[SEP]
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)
# 统一长度为7
tokens = tokens + ['[PAD]'] + ['[PAD]']
print(tokens)
# 获取注意力掩码
attention_mask = [1 if i != '[PAD]' else 0 for i in tokens]
print(attention_mask)
# 获取标记ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

`unsqueeze()` 的目的是改变张量的形状，增加一个维度。下面两个都将变为 1 * 7 的二维矩阵。

In [ ]:
# 将注意力掩码和标记ID转化为张量，方便后续计算
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = 1 - torch.tensor(attention_mask).unsqueeze(0)

#### 获取嵌入向量

In [ ]:
hidden_rep, cls_head = model(token_ids, attention_mask = attention_mask)
print(hidden_rep.shape)
print(cls_head.shape)

第1个值`hidden_rep`表示隐藏状态的特征，它包括从顶层编码器（编码器12）获得的所有标记的特征。第2个值`cls_head`表示`[CLS]`标记的特征
- `hidden_rep[0][0]`给出了第1个标记[CLS]的特征。
- `hidden_rep[0][1]`给出了第2个标记`I`的特征。
- `hidden_rep[0][2]`给出了第3个标记`love`的特征。

`cls_head`作为句子I love Paris的整句特征

### BERT的所有编码器层获得嵌入

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

下载预训练的BERT模型和词元分析器

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True) # 允许我们从所有编码层获得嵌入
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#### 输入预处理

In [ ]:
sentence = 'I love Paris'
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens = tokens + ['[PAD]'] + ['[PAD]']

attention_mask = [1 if i != '[PAD]' else 0 for i in tokens]
token_ids = tokenizer.convert_tokens_to_ids(tokens)

token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

#### 获取嵌入

In [ ]:
last_hidden_state, pooler_output, hidden_states = model(token_ids, attention_mask = attention_mask)

- `last_hidden_state`包含从最后的编码器（编码器12）中获得的所有标记的特征。
- `pooler_output`表示来自最后的编码器的[CLS]标记的特征，它被一个线性激活函数和tanh激活函数进一步处理。
- `hidden_states`包含从所有编码器层获得的所有标记的特征

In [ ]:
print(last_hidden_state.shape)

- `last_hidden_state[0][0]`给出了第1个标记[CLS]的特征。
- `last_hidden_state[0][1]`给出了第2个标记`I`的特征。
- `last_hidden_state[0][2]`给出了第3个标记`love`的特征。

In [ ]:
print(pooler_output.shape)

`pooler_output`作为句子I love Paris的整句特征

In [ ]:
len(hidden_states)

- `hidden_states[0]`包含从输入嵌入层获得的所有标记的特征。
- `hidden_states[1]`包含从第1个编码器层获得的所有标记的特征。
- `hidden_states[2]`包含从第2个编码器层获得的所有标记的特征。
- `hidden_states[12]`包含从最后一个编码器层获得的所有标记的特征。

In [ ]:
print(hidden_states[0].shape)
print(hidden_states[1].shape)